In [3]:
import pandas as pd
import numpy as np
import os

In [237]:
usecols = ["Chr", "Start", "End", "Ref", "Alt", "Func.refGene", "Otherinfo10", "Otherinfo11"]
dtype = {"Chr":"category",
          "Start":np.int32,
          "End":np.int32,
          "Ref":"category",
          "Alt":"category",
          "Func.refGene":"category",
          "Otherinfo10":"category",
          "Otherinfo11":"string"}

In [238]:
df = pd.read_csv("anno1.fixed.chrY.genes.VE_22251.hg38_multianno.csv", usecols=usecols, dtype=dtype)
df = pd.eval("DP = df.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=df)
df = pd.eval("AF = df.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=df)

In [60]:
# this works on Chr, Ref, Alt, Func.refGene, Otherinfo10
def category_filters(**kwargs):
    new_df = df
    for key, value in kwargs.items():
        if key == "FuncrefGene":
            new_df = new_df[new_df["Func.refGene"] == value]
        else:
            new_df = new_df[new_df[key] == value]
    return new_df

In [7]:
# for Func.refGene, enter 'FuncrefGene' instead 
display(category_filters(FuncrefGene="exonic", Otherinfo10="PASS"))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
0,chrY,2787119,2787119,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=415;VD=7;AF=0.0169...,415,0.0169
1,chrY,2787445,2787445,T,C,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=317;VD=4;AF=0.0126...,317,0.0126
15,chrY,2865200,2865200,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=173;VD=2;AF=0.0116...,173,0.0116
16,chrY,2866887,2866887,G,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=232;VD=4;AF=0.0172...,232,0.0172
28,chrY,2961350,2961350,G,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=152;VD=2;AF=0.0132...,152,0.0132
33,chrY,2978846,2978846,T,C,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=182;VD=2;AF=0.011;...,182,0.0110
34,chrY,2979004,2979004,A,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=148;VD=4;AF=0.027;...,148,0.0270
37,chrY,2979454,2979454,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=232;VD=10;AF=0.043...,232,0.0431
43,chrY,5100327,5100327,G,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=88;VD=88;AF=1;BIAS...,88,1.0000
44,chrY,5100614,5100614,T,G,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=189;VD=189;AF=1;BI...,189,1.0000


In [8]:
# this assumes greater than i.e 'DP = 15' means keep DP vals > 15
def num_filter(**kwargs):
    new_df = df
    for key, value in kwargs.items():
        new_df.loc[:,key].where(new_df.loc[:,key] > value,inplace=True)
    return new_df.dropna()

In [9]:
display(num_filter(DP=150,AF=.15))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
44,chrY,5100614,5100614,T,G,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=189;VD=189;AF=1;BI...,189.0,1.0


In [294]:
def subtraction(df1, df2):
    uqid_df1 = df1[df1.columns[1:4]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()
    uqid_df2 = df2[df2.columns[1:4]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()  
    uqid = pd.concat([uqid_df1, uqid_df2], ignore_index=True)
    
    uqid_df = pd.DataFrame({'0':np.zeros(len(uqid))}, index=uqid)
    uqid_df.reset_index().sort_index().groupby('index').filter(lambda x: len(x) == 1)
    # uqid_df.rename(columns={'index':'unique ids'}, inplace=True)
    # uqid_df.drop('0', axis=1, inplace=True)
    return uqid_df

In [295]:
df2_ = pd.read_csv("chrY_22243.csv", usecols=["Chr", "Start", "End", "Ref", "Alt"], sep='\t')
subtraction(df, df2_)

,index,0
0,27871192787119C,0.0
1,27874452787445T,0.0
2,27875692787569A,0.0
3,27876242787624C,0.0
4,28418922841892T,0.0
...,...,...
824,2503830225038302T,0.0
825,2503888125038881G,0.0
826,2503891925038919G,0.0
827,2562235925622359C,0.0


In [216]:
# if sub algorithm works, return df should have length = len(df)//2 - 3
display(gen_unique_id(df[["Start","End", "Ref", "Alt"]]))
# print(sub_v-p(df[len(df)//2 + 3:], df[:len(df)//2]))
# output is df1 w indices of where it is equal

In [210]:
#TODO: make option to see what is NA and what is not NA
#TODO: make program and functions run on a class
# possible error of pd.to_numeric int out of range may come from long ref/alt strings